In [16]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pickle
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [17]:
df = pd.read_csv('spam.csv', encoding='latin-1')
# Keeping only the neccessary columns
df = df[['v2','v1']]

In [18]:
df.head()

,v2,v1
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [19]:
df['v2'] = df['v2'].apply(lambda x: x.lower())
df['v2'] = df['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [20]:
df.head()

,v2,v1
0,go until jurong point crazy available only in ...,ham
1,ok lar joking wif u oni,ham
2,free entry in 2 a wkly comp to win fa cup fina...,spam
3,u dun say so early hor u c already then say,ham
4,nah i dont think he goes to usf he lives aroun...,ham


In [21]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['v2'].values)
X = tokenizer.texts_to_sequences(df['v2'].values)
X = pad_sequences(X)

In [22]:
embed_dim = 128
lstm_out = 196

In [27]:
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

In [28]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(df['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [29]:
# tensorboard = TensorBoard(log_dir="logs/icp5", histogram_freq=1, write_graph=True, write_images=False)

In [34]:
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, batch_size=batch_size, epochs=2,verbose=2)

Epoch 1/2
 - 51s - loss: 0.1697 - acc: 0.9419
Epoch 2/2
 - 51s - loss: 0.0399 - acc: 0.9885


In [35]:
score, acc = model.evaluate(X_test, Y_test, verbose=2, batch_size=batch_size)
print(model.metrics_names[0],":",score)
print(model.metrics_names[1],":",acc)

loss : 0.06811000676162221
acc : 0.9847743338771071
